In [69]:
import pandas as pd
covid_loc='https://github.com/g-raceyoung/PUBPOL542/blob/main/biweekly-counts-rates-by-geography-dec-29.xlsx?raw=true'
covid_cases=pd.read_excel(covid_loc, sheet_name='Census')

# at census tract level (11 digits)

In [70]:
park_visit_loc = 'https://github.com/g-raceyoung/PUBPOL542/raw/main/2020-4-cleaned.csv' #cleaned data
park_visit = pd.read_csv(park_visit_loc)

# at census block group level (12 digits, need to drop last digit)
drop_last = lambda s : int(str(s)[:-1])

park_visit['cbgs_id'] = park_visit['cbgs_id'].apply(drop_last) #removing the block group level designator to get to tract level

park_visit['week'] = pd.to_datetime(park_visit['week']) #converting 'week' from string to datetime64[ns]

In [116]:
park_visit = park_visit.groupby(['cbgs_id', 'week'], as_index=False).sum('count') #grouping by tract_id and summing over count
park_visit.columns = ['cbgs_id','week_collected','visitor_count'] #renaming columns
park_visit.head()

,cbgs_id,week_collected,visitor_count
0,53001950100,2020-04-27,8
1,53001950200,2020-04-27,4
2,53001950300,2020-04-27,28
3,53001950400,2020-04-27,16
4,53001950500,2020-04-27,42


In [132]:
# filter for weeks prior to park counts collected
covid_cases_pre = covid_cases[covid_cases['Week_Start'] <= park_visit['week_collected'][0]]

# summing up the number of covid cases per tract prior to the date where park counts were collected
covid_cases_pre = covid_cases_pre.groupby(['Census'], as_index=False)['Positives'].sum()
covid_cases_pre.columns = ['census','precount_positive']

# trying also summing all cases to date because having a lot of records with 0 is not good for later regression analysis
covid_cases_all = covid_cases.groupby(['Census'], as_index=False)['Positives'].sum()
covid_cases_all.columns = ['census','all_positive'] 

In [129]:
age_loc = 'https://raw.githubusercontent.com/g-raceyoung/PUBPOL542/main/Median_age_by_sex.csv'
age = pd.read_csv(age_loc)
age = age.drop(0) #removing first row
age.reset_index(drop=True, inplace=True) #resetting index to start at 0 again
age = age.iloc[:,[0,2]] #selecting relevant columns
age.columns = ['geo_id', 'median_age'] #renaming columns

drop_front = lambda s : int(s[9:])
age['geo_id'] = age['geo_id'].apply(drop_front) #dropping irrelevant part of geo_id
age['median_age'] = pd.to_numeric(final['median_age'], errors='coerce') #converting age from string to float, generates NaN where the string is non-numeric (i.e. "-")
age = age.dropna() #dropping these newly created NaNs
age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 905 entries, 0 to 904
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   geo_id      905 non-null    int64  
 1   median_age  905 non-null    float64
dtypes: float64(1), int64(1)
memory usage: 21.2 KB


In [131]:
# MERGING

# parks & age
join_1 = park_visit.merge(age, left_on='cbgs_id', right_on='geo_id', how='inner')
join_1.head()

# adding covid_cases_pre
join_2 = join_1.merge(covid_cases_pre, left_on='geo_id', right_on='census', how='inner')
join_2.head()

# adding covid_cases_all
final = join_2.merge(covid_cases_all, left_on='geo_id', right_on='census', how='inner')
final.head()

final.drop(['geo_id','census_x','census_y','week_collected'], axis=1, inplace=True)

final.head()

,cbgs_id,visitor_count,median_age,precount_positive,all_positive
0,53001950400,16,32.9,0,0
1,53005010100,46,40.7,0,0
2,53005010201,71,40.3,0,0
3,53005010202,53,49.7,0,0
4,53005010300,60,40.4,0,0
